What to do ?
- single dimension input
- Model class implemented
- Model class method `compile` implemented

In [13]:
import numpy as np

In [14]:
input_ = np.random.randn(5)

In [15]:
input_

array([-1.40346088, -1.84182672,  0.63988283,  0.77701001,  0.05279381])

In [16]:
class Input:
    
    '''
    making it for only one dimensional input
    '''
    def __init__(self, input_shape:int, layer_name:str):
        
        # self.input_shape = input_shape      
        self.layer_name = layer_name
        self.units = input_shape
        self.input_data:np.array = np.zeros(self.units) # None only at time of initialization
        
        # these parameters doesnt exist for input layer
        self.previous_layer = None
        self.previous_layer_units = None
        
        
    def forward_pass(self):
        print("at layer      >>>>>>>>>>     ",self.layer_name)
        return self.input_data

In [17]:
class Dense:
    
    def __init__(self, units:int, activation:str, layer_name:str):
        self.units = units
        self.activation = activation
        self.layer_name = layer_name
        self.previous_layer:Dense = None
        self.previous_layer_values = None
        self.previous_layer_units = None
        
        
    
    def forward_pass(self) -> np.array:
        
        '''
        jab ek layer k liye forward pass chalega toh 
        wo previous layer ki values ko fetch karega and 
        calculation krke apni nayi values send karega
        '''
        #  determining previous layer values
        
        self.previous_layer_units = self.previous_layer.units
        
        
        if isinstance(self.previous_layer, Input):
            self.previous_layer_values = self.previous_layer.input_data

        
        
        if isinstance(self.previous_layer, Dense):
            self.previous_layer_values = self.previous_layer.activation_output

    
        
                
        # initializing weights and biases of this layer
        self.__weights, self.__biases = self.__initialize_weights_and_biases(self.previous_layer_units)
        
        
        # calculating output ---> WiXi + Bo 
        self.output = np.add(np.dot(self.__weights, self.previous_layer_values), self.__biases)

        
        if self.activation.lower() == 'relu':
            activation_func = np.vectorize(self.__relu_activation)
            self.activation_output = activation_func(self.output).astype(np.float32)
            
        elif self.activation.lower() == 'sigmoid':
            activation_func = np.vectorize(self.__sigmoid_activation)
            self.activation_output = activation_func(self.output).astype(np.float32)
            
        else: # linear
            self.activation_output = self.output
            
        
        print("\n\ncurrent_layer_units = ",self.units)
        print("biases_shape = ",self.__biases.shape)
        print("weights_shape = ",self.__weights.shape)
        print("previous layer units = ",self.previous_layer_units)
        print("previous layer values = ",self.previous_layer_values)
        print("Output for next layer is =", self.activation_output)
        
        
        return self.activation_output

        
    def __initialize_weights_and_biases(self,previous_layer_units:int):
        
        '''
        Note: using `he normal` initialization
        '''
        
        biases = np.random.randn(self.units)*np.sqrt(2/previous_layer_units)
        weights = np.random.randn(self.units, previous_layer_units)*np.sqrt(2/previous_layer_units)
        return weights, biases
        

    def __relu_activation(self, layer_output:np.array):
        return max(0, layer_output)
    
    
    def __sigmoid_activation(self, layer_output:np.array):
        if layer_output>0:
            return 1/(1+np.exp(-layer_output)) # 1/(1+e^-z)
        else:
            return np.exp(layer_output)/(1+np.exp(layer_output)) # e^z / (1+e^z)
    

In [18]:
class Model:
    
    def __init__(self):
        self.layers = []
        self.count = 1
        
               
    def add(self, layer:Dense):
        layer.previous_layer = None
        '''
        set last item in list as previous layer
        '''
        self.layers.append(layer)
        # if self.count>1:
        #     layer.previous_layer = self.layers[-1]
            
            
        # if isinstance(layer, (Dense, Input)):
        #     self.layers.append(layer) # storing the layer object
        #     self.count += 1
        
        # return self.layers
    def show_layers(self):
        print("=================== LAYER INFO ===================")
        for index, layer in enumerate(self.layers):
            print(f"""
                  Index::{index}
                        Layer Name:: {layer.layer_name} {type(layer.layer_name)}
                        Layer Address:: {layer}  {type(layer)}      
                        Layer units::{layer.units} {type(layer.units)}
                        Previous Layer::{layer.previous_layer} {type(layer.previous_layer)}
                        
                        """)
                
            
    def compile_(self, input_data:np.array): 
        '''
        connecting all the layers
        
        running forward pass for all layers
        
        and connect input layer with input data
        '''       
        
        #  connecting all the layers
        
        for i in range(len(self.layers)-1):
            print(f"set {self.layers[i].layer_name}::{self.layers[i]} as previous layer of {self.layers[i+1].layer_name}::{self.layers[i+1]}")
            self.layers[i+1].previous_layer = self.layers[i]

        print("\n\nDONE WITH CONNECTING ALL THE LAYERS \n\n")
        
        if isinstance(self.layers[0], Input) and self.layers[0].units == input_data.shape[0]:
            self.layers[0].input_data = input_data
            print("done with input layer!!")
        # else:
            # raise ValueError("Input layer shape ({}) is not same as input data shape({})".format(self.layers[0].input_shape, input_data.shape))
                
        for layer in self.layers:
            
            layer.forward_pass()
            print(f"done forward pass for {layer.layer_name}")
            


In [19]:
model= Model()

In [20]:
model.add(Input(input_shape=5, layer_name="input_"))
model.add(Dense(units = 10,activation = "sigmoid", layer_name="first"))
model.add(Dense(units = 20,activation = "sigmoid", layer_name="second"))
model.add(Dense(units = 1,activation = "sigmoid", layer_name="third"))

In [21]:
model.compile_(input_)
# check dtype of input object

set input_::<__main__.Input object at 0x73838022be20> as previous layer of first::<__main__.Dense object at 0x73838022a200>
set first::<__main__.Dense object at 0x73838022a200> as previous layer of second::<__main__.Dense object at 0x738380228d60>
set second::<__main__.Dense object at 0x738380228d60> as previous layer of third::<__main__.Dense object at 0x73838022bb80>


DONE WITH CONNECTING ALL THE LAYERS 


done with input layer!!
at layer      >>>>>>>>>>      input_
done forward pass for input_


current_layer_units =  10
biases_shape =  (10,)
weights_shape =  (10, 5)
previous layer units =  5
previous layer values =  [-1.40346088 -1.84182672  0.63988283  0.77701001  0.05279381]
Output for next layer is = [0.8049293  0.04211444 0.74088365 0.43989772 0.04905093 0.51171005
 0.8193818  0.7061816  0.7506211  0.08681619]
done forward pass for first


current_layer_units =  20
biases_shape =  (20,)
weights_shape =  (20, 10)
previous layer units =  10
previous layer values =  [0.8049293  0

In [22]:
input_

array([-1.40346088, -1.84182672,  0.63988283,  0.77701001,  0.05279381])

In [23]:
model.show_layers()

=================== LAYER INFO ===================

                  Index::0
                        Layer Name:: input_ <class 'str'>
                        Layer Address:: <__main__.Input object at 0x73838022be20>  <class '__main__.Input'>      
                        Layer units::5 <class 'int'>
                        Previous Layer::None <class 'NoneType'>
                        
                        

                  Index::1
                        Layer Name:: first <class 'str'>
                        Layer Address:: <__main__.Dense object at 0x73838022a200>  <class '__main__.Dense'>      
                        Layer units::10 <class 'int'>
                        Previous Layer::<__main__.Input object at 0x73838022be20> <class '__main__.Input'>
                        
                        

                  Index::2
                        Layer Name:: second <class 'str'>
                        Layer Address:: <__main__.Dense object at 0x738380228d60>  <cl

In [24]:
input_

array([-1.40346088, -1.84182672,  0.63988283,  0.77701001,  0.05279381])